# Univariate and Multivariate Statistics with NHANES Data



## Data Wrangling

Data wrangling is the process of transforming and mapping data from a raw data form into another format with the intent of making it more appropriate and valuable for data analytics.

1. Data Discovery
    
    This all-encompassing term describes how to understand your data. This is the first step to familiarize yourself with your data. You will use the Data Dictionary and different PANDAS commands for this step:
    

2. Structuring 
    
    The next step is to organize the data. Raw data is typically unorganized and much of it may not be useful for the end product. This step is important for easier computation and analysis in the later steps.
    

3. Cleaning 
    
    There are many different forms of cleaning data, for example one form of cleaning data is catching dates formatted in a different way and another form is removing outliers that will skew results and also formatting null values. This step is important in assuring the overall quality of the data.
    

4. Enriching 
    
    At this step determine whether or not additional data would benefit the data set that could be easily added.
    




We will use PANDAS to complete these data wrangling steps.

### PANDAS First Steps Import

 PANDAS is a module of code that can be used in python.  To use this module or any other module you have to import the module.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats

#set pandas to display all columns
pd.set_option('display.max_columns', None)

#set pandas to display 250 rows
pd.set_option('display.max_rows', 250)

In [ ]:
df = pd.read_csv("assets/modified_NHANES.csv")

# Data Discovery

### Viewing your data

The first thing to do when opening a new dataset is print out a few rows to keep as a visual reference. We accomplish this with `.head()`

df.head() outputs the first five rows of your DataFrame by default, but we could also pass a number as well: df.head(20) would output the top twenty rows.

To see the last five rows use .tail(). tail() also accepts a number to display the desired last rows.

In [ ]:
df.head(20)

In [ ]:
df.shape

### Data Dictionary

We need to determine how each column is coded.  

- Do the columns contain continuous data or catagorical data?  
- Which of these columns are important to our project question or hypothesis?
- How do we need to process our data to be usable (if it is a catagorical column  of 1-6, 9. What do those numbers mean?

You can open the NHANES Abbreviated Data Dictionary Word file in the assets directory.

# Structuring and Cleaning Data

### Handling duplicates

Your dataset should not have duplicate rows, but it is always important to verify you aren't aggregating duplicate rows.  

`df.duplicated()` returns whether or not the row is duplicated

`df.duplicated().sum()` returns the number or rows duplicated

`df[df.duplicated()]` return the duplicated rows



In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

This dataset did not have duplicated rows.  If it did you would run 

`df = df.drop_duplicates()` to remove the duplicated rows. 

`.info()` provides the essential details about your dataset, such as the number of rows and columns, the number of non-null values, what type of data is in each column, and how much memory your DataFrame is using. 

Notice in our movies dataset we have some obvious missing values in the `PAT_CTY_CODE` and `SEX_CODE` columns as well as others. We'll look at how to handle those in a bit.

Seeing the datatype quickly is actually quite useful.  Calling `.info()` will quickly point out that your column you thought was all integers are actually string objects.


In [ ]:
df.info()

### How to work with missing values

When exploring data, you’ll most likely encounter missing or null values, which are essentially placeholders for non-existent values. Most commonly you'll see Python's `None` or NumPy's `np.nan`, each of which are handled differently in some situations.

There are two options in dealing with nulls: 

1. Get rid of rows or columns with nulls
2. Replace nulls with non-null values, a technique known as **imputation**

Let's calculate to total number of nulls in each column of our dataset. The first step is to check which cells in our DataFrame are null:

In [ ]:
df.isnull().sum()

In the variable DMDEDUC2, we can look at the counts of each code.

In [ ]:
df.DMDEDUC2.value_counts()

In [ ]:
print(df.DMDEDUC2.value_counts().sum())
print(1621 + 1366 + 1186 + 655 + 643 + 3) # Manually sum the frequencies
print(df.shape)

We can determine the number of entries that are missing or Null

In [ ]:
pd.isnull(df.DMDEDUC2).sum()

### Recoding Data

We can recode the data preventing the loss of data from other columns.  For example, we can recode DMDEDUC2.  Look at the data dictionary that describes how this is coded.  

First we will recode the data that is not missing so that it is easier to read.  We will recode the data into a new column `DMDEDUC2x`

In [ ]:
df["DMDEDUC2x"] = df.DMDEDUC2.replace({1: "<9", 2: "9-11", 3: "HS/GED", 4: "Some college/AA", 5: "College", 
                                       7: "Refused", 9: "Don't know"})
df.DMDEDUC2x.value_counts()

Now we will do replace missing or null data with the term 'missing'

In [ ]:
df["DMDEDUC2x"] = df.DMDEDUC2x.fillna("Missing")

print(df.DMDEDUC2x.value_counts().sum())
print(df.shape)

Now recode RIAGENDR to RIAGENDRx

In [ ]:
df["RIAGENDRx"] = df.RIAGENDR.replace({1: "Male", 2: "Female"})

In [ ]:
df


In [ ]:
df.DMDHHSIZ.value_counts()

### Removing rows that have missing data that are important for our project - Height, Weight , BMI

We can describe the data that is not missing

In [ ]:
df.BMXWT.dropna().describe()

In [ ]:
sns.displot(df.BMXWT.dropna())
plt.show()

In [ ]:
sns.displot(df.BPXSY1.dropna())
plt.show()

In [ ]:
bp = sns.boxplot(data=df.loc[:, ["BPXSY1", "BPXSY2", "BPXDI1", "BPXDI2"]])
_ = bp.set_ylabel("Blood pressure in mm/Hg")

In [ ]:
df["agegrp"] = pd.cut(df.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80]) # Create age strata based on these cut points
plt.figure(figsize=(12, 5))  # Make the figure wider than default (12cm wide by 5cm tall)
sns.boxplot(x="agegrp", y="BPXSY1", data=df)  # Make boxplot of BPXSY1 stratified by age group
plt.show()

In [ ]:
df["agegrp"] = pd.cut(df.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
plt.figure(figsize=(12, 5))
sns.boxplot(x="agegrp", y="BPXSY1", hue="RIAGENDRx", data=df)
plt.show()

In [ ]:
df["agegrp"] = pd.cut(df.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
plt.figure(figsize=(12, 5))
sns.boxplot(x="RIAGENDRx", y="BPXSY1", hue="agegrp", data=df)
plt.show()

In [ ]:
df.groupby("agegrp")["DMDEDUC2x"].value_counts()

In [ ]:
dx = df.loc[~df.DMDEDUC2x.isin(["Don't know", "Missing"]), :]  # Eliminate rare/missing values
dx = dx.groupby(["agegrp", "RIAGENDRx"])["DMDEDUC2x"]
dx = dx.value_counts()
dx = dx.unstack() # Restructure the results from 'long' to 'wide'

dx = dx.apply(lambda x: (x/x.sum())*100, axis=1) # Normalize within each stratum to get proportions
print(dx.to_string(float_format="%.3f"))  # Limit display to 3 decimal places

# Multivariate Statistics

In [ ]:
sns.regplot(x="BMXLEG", y="BMXARML", data=df, fit_reg=False, scatter_kws={"alpha": 0.2})
plt.show()

In [ ]:
sns.FacetGrid(df, col="RIAGENDRx").map(plt.scatter, "BMXLEG", "BMXARML", alpha=0.4).add_legend()
plt.show()